### Programita de prueba para visualizar redes dinámicas

In [7]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import os
import ffmpeg

plt.rcParams['text.usetex'] = True

In [205]:
# Numero de nodos
n = 200
# Pesos de los nodos
w = {i: np.random.exponential(scale=1) for i in range(n)}
# Semilla (tiene que ser un int)
seed = np.random.randint(1000)
theta_mul = 100

# Red multiplicativa
GTG = nx.geographical_threshold_graph(n, theta=theta_mul, weight=w, 
                                      dim=2, seed=seed, additive=False)

posi = {i: GTG.nodes[i]['pos'] for i in range(n)}
print("Conectividad multiplicativo: {:.2f}".format(np.mean(list(dict(nx.degree(GTG)).values()))))

# Descomentar para graficar solo este
#plt.figure(dpi=100)
#nx.draw(GTG, node_size=25*np.array(list(w.values())), width=.2, pos=posi)

Conectividad multiplicativo: 5.17


In [165]:
def update_graph(graph, new_w, update_links=False, savefig=False, filename=''):
    nx.set_node_attributes(GTG, dict(enumerate(w)), 'weight')
    

In [130]:
figpath = os.path.join(os.getcwd(), 'data', "3_dynamic_gtg", "figs")
savefig = True

fig = plt.figure(dpi=100)

for i in range(400):
    dw = np.random.normal(1, .1, size=n)

    w = {i: w[i]*dw[i] for i in range(n)}
    a = np.array(list(w.values()))
    
    update_graph(GTG, w, update_links=False, savefig=False)

    if i % 10 == 0:
        GTG = nx.geographical_threshold_graph(n, theta=theta_mul, weight=w, 
                                              dim=2, pos=posi, additive=False)

    if savefig:
        filename = os.path.join(figpath, 'test_{:03d}.png'.format(i))
        ax = plt.gca()
        ax.set_title('test_{}.png'.format(i))
        nx.draw(GTG, node_size=25*a, width=.2, pos=posi, ax=ax)
        #plt.tight_layout()
        plt.savefig(filename, format="PNG")
        plt.clf()
        

<Figure size 640x480 with 0 Axes>

In [206]:
figpath = os.path.join(os.getcwd(), 'data', "3_dynamic_gtg", "figs")
savefig = True

fig = plt.figure(figsize=(8, 6), dpi=100)
#ax = plt.subplot(111, aspect = 'equal')
#plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)


for i in range(400):
    dw = np.random.normal(1, .1, size=n)

    w = {i: w[i]*dw[i] for i in range(n)}
    a = np.array(list(w.values()))
    
    update_graph(GTG, w, update_links=False, savefig=False)

    if i % 10 == 0:
        GTG = nx.geographical_threshold_graph(n, theta=theta_mul, weight=w, 
                                              dim=2, pos=posi, additive=False)
    
    if savefig:
        filename = os.path.join(figpath, 'test_{:03d}.png'.format(i))
        #plt.tight_layout()
        ax = plt.gca()
        ax.set_xlim([-0, 1])
        ax.set_ylim([-0, 1])
        ax.set_title('test_{}'.format(i))
        nx.draw(GTG, node_size=25*a, width=.2, pos=posi, ax=ax)
        plt.savefig(filename, format="PNG")
        plt.clf()

<Figure size 800x600 with 0 Axes>

In [102]:
def delete_images(figpath):
    for file in os.listdir(figpath):
        print(file)
        if file.endswith('.png'):
            os.remove(os.path.join(figpath, file))  

In [207]:
(
    ffmpeg
    .input(os.path.join(figpath, 'test_*.png'), pattern_type='glob', framerate=15)
    .output(os.path.join(figpath, 'movie.mp4'))
    .overwrite_output()
    .run(quiet=True)
)

delete_images(figpath)

test_279.png
test_350.png
test_239.png
test_001.png
test_084.png
test_273.png
test_367.png
test_392.png
test_270.png
test_160.png
test_012.png
test_231.png
movie.mp4
test_262.png
test_141.png
test_397.png
test_376.png
test_280.png
test_162.png
test_063.png
test_206.png
test_143.png
test_065.png
test_360.png
test_011.png
test_146.png
test_217.png
test_171.png
test_307.png
test_122.png
test_062.png
test_303.png
test_003.png
test_002.png
test_216.png
test_047.png
test_173.png
test_344.png
test_396.png
test_342.png
test_313.png
test_235.png
test_167.png
test_018.png
test_354.png
test_373.png
test_264.png
test_305.png
test_382.png
test_189.png
test_363.png
test_103.png
test_277.png
test_155.png
test_255.png
test_176.png
test_071.png
test_053.png
test_335.png
test_395.png
test_192.png
test_158.png
test_017.png
test_257.png
test_294.png
test_142.png
test_269.png
test_349.png
test_391.png
test_013.png
test_372.png
test_129.png
test_241.png
test_332.png
test_154.png
test_394.png
test_072.png
te